# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,23.64,97,27,2.68,EC,1695091322
1,1,bengkulu,-3.8004,102.2655,26.73,76,91,4.72,ID,1695091322
2,2,tsiombe,-25.3000,45.4833,19.51,97,42,2.31,MG,1695091322
3,3,punta arenas,-53.1500,-70.9167,1.06,64,0,4.12,CL,1695091323
4,4,dudinka,69.4058,86.1778,2.57,87,100,5.70,RU,1695091323


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

C:\Users\Jared\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[city_data_df['Max Temp']>21]
city_data_df = city_data_df[city_data_df['Max Temp']<27]
city_data_df = city_data_df[city_data_df['Wind Speed']<4.50]
city_data_df = city_data_df[city_data_df['Cloudiness']==0]

# Drop any rows with null values
city_data_df = city_data_df.dropna(how='any',axis=0)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,itaocara,-21.6692,-42.0761,21.46,80,0,0.67,BR,1695091360
139,139,springhill,33.0060,-93.4668,23.94,48,0,2.32,US,1695091365
164,164,mccook,40.2019,-100.6257,24.25,51,0,4.12,US,1695091373
166,166,morehead city,34.7229,-76.7260,21.67,87,0,0.00,US,1695091373
213,213,al bawiti,28.3492,28.8659,23.32,52,0,2.25,EG,1695091388
214,214,stanthorpe,-28.6667,151.9500,22.46,44,0,2.34,AU,1695091388
286,286,remire-montjoly,4.9167,-52.2667,26.02,94,0,1.03,GF,1695091411
309,309,carutapera,-1.1950,-46.0200,25.80,87,0,3.33,BR,1695091418
314,314,denov,38.2667,67.9000,23.67,24,0,0.45,UZ,1695091420
323,323,shunyi,40.1250,116.6453,22.97,49,0,4.02,CN,1695091423


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
header_list = ['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name']
hotel_df = hotel_df.reindex(columns = header_list)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
123,itaocara,BR,-21.6692,-42.0761,80,NaN
139,springhill,US,33.0060,-93.4668,48,NaN
164,mccook,US,40.2019,-100.6257,51,NaN
166,morehead city,US,34.7229,-76.7260,87,NaN
213,al bawiti,EG,28.3492,28.8659,52,NaN
214,stanthorpe,AU,-28.6667,151.9500,44,NaN
286,remire-montjoly,GF,4.9167,-52.2667,94,NaN
309,carutapera,BR,-1.1950,-46.0200,87,NaN
314,denov,UZ,38.2667,67.9000,24,NaN
323,shunyi,CN,40.1250,116.6453,49,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "filter":"",
    "bias":"",
    "limit":limit,
    "apiKey":geoapify_key
}

# Set apiKey to our geoapify_key
params["apiKey"] = geoapify_key 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
itaocara - nearest hotel: Hotel Itaocara
springhill - nearest hotel: No hotel found
mccook - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
al bawiti - nearest hotel: Old Oasis Hotel
stanthorpe - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
carutapera - nearest hotel: No hotel found
denov - nearest hotel: Turon Hotel
shunyi - nearest hotel: 会议楼
fort smith - nearest hotel: Red Roof Inn
benghazi - nearest hotel: ترجاب
at taj - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
gillette - nearest hotel: Montgomery Hotel
mao - nearest hotel: Eurohotel
grand island - nearest hotel: Grand Island Travelodge
glen innes - nearest hotel: No hotel found
artemisa - nearest hotel: Campoamor
idri - nearest hotel: No hotel found
juhaynah - nearest hotel: No hotel found
watford city - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mari

,City,Country,Lat,Lng,Humidity,Hotel Name
123,itaocara,BR,-21.6692,-42.0761,80,Hotel Itaocara
139,springhill,US,33.0060,-93.4668,48,No hotel found
164,mccook,US,40.2019,-100.6257,51,No hotel found
166,morehead city,US,34.7229,-76.7260,87,"Bask Hotel Big Rock Landing, Trademark Collect..."
213,al bawiti,EG,28.3492,28.8659,52,Old Oasis Hotel
214,stanthorpe,AU,-28.6667,151.9500,44,No hotel found
286,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
309,carutapera,BR,-1.1950,-46.0200,87,No hotel found
314,denov,UZ,38.2667,67.9000,24,Turon Hotel
323,shunyi,CN,40.1250,116.6453,49,会议楼


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)